In [1]:
import glob
import sys
import os
import nibabel as nib
import numpy as np
import tifffile
from subprocess import call

In [2]:
warp_list=glob.glob('/mnt/d/ForANTs/*1Warp.nii.gz')
del(warp_list[0])
warp_list

['/mnt/d/ForANTs/RegToTemplate_20200917051Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20200917061Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20200924071Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20200924081Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20200924091Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20200924101Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20200924111Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201001131Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201001151Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201001161Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201001171Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201022191Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201022201Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201022211Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201022231Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201022241Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201116261Warp.nii.gz',
 '/mnt/d/ForANTs/RegToTemplate_20201116271Warp.n

In [4]:
warp_list[2].split('1Warp.nii.gz')[0]+'_Jacobian.nii'

'/mnt/d/ForANTs/RegToTemplate_2020091706_Jacobian'

In [5]:
for name in warp_list:
    output_name = name.split('1Warp.nii.gz')[0]+'_Jacobian.nii.gz'
    #Compute the jacobian determinant of all the warps
    job_string = "CreateJacobianDeterminantImage 3 "+name+" "+os.path.basename(output_name)+" 1 0"    
    call([job_string],shell=True)

In [22]:
import nibabel as nib
import numpy as np
img_to_cat=np.zeros([len(warp_list),1280,1080,251])
for img_nb,name in enumerate(warp_list):
    img=nib.load(name.split('1Warp.nii.gz')[0]+'_Jacobian.nii.gz')
    img_to_cat[img_nb,:,:,:]=img.get_fdata()

In [4]:
genotype=np.array([0,0,0,1,1,1,0,1,0,1,0,1,1,0,1,1,0,1,0,1,0])
len(genotype)

21

In [ ]:
img_WT=np.mean(img_to_cat[genotype==0,:,:,:],axis=0)
img_WT_std=np.std(img_to_cat[genotype==0,:,:,:],axis=0)
img_FMR=np.mean(img_to_cat[genotype==1,:,:,:],axis=0)
img_FMR_std=np.std(img_to_cat[genotype==1,:,:,:],axis=0)
nib.save(img_WT,'/mnt/mnt/d/ForANTs/MeanJacobWT.nii.gz')
nib.save(img_WT_std,'/mnt/d/ForANTs/StdJacobWT.nii.gz')
nib.save(img_FMR,'/mnt/d/ForANTs/MeanJacobSc1n.nii.gz')
nib.save(img_FMR_std,'/mnt/d/ForANTs/StdJacobSc1n.nii.gz')

/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:3256: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/lib/python3/dist-packages/numpy/core/_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/usr/lib/python3/dist-packages/numpy/core/_methods.py:216: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/lib/python3/dist-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/usr/lib/python3/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [55]:
np.where(genotype)

(array([ 3,  4,  5,  7,  9, 11, 12, 14, 15, 17, 19]),)

In [42]:
len(np.where(np.invert(genotype.astype(bool)))[0])

10

In [5]:
img_to_cat=np.zeros([len(np.where(np.invert(genotype.astype(bool)))[0]),1280,1080,251])
for img_nb in range(0,len(np.where(np.invert(genotype.astype(bool)))[0])):
    if not img_nb==3:
        img=nib.load(warp_list[np.where(np.invert(genotype.astype(bool)))[0][img_nb]].split('1Warp.nii.gz')[0]+'_Jacobian.nii.gz')
        img_to_cat[img_nb,:,:,:]=img.get_fdata()


In [6]:
img_WT.shape

NameError: name 'img_WT' is not defined

In [7]:
img_to_cat.shape

(10, 1280, 1080, 251)

In [15]:
img_WT=np.mean(img_to_cat[[0,1,2,4,5,6,7,8,9],:,:,:],axis=0)
tifffile.imsave('/mnt/d/ForANTs/MeanJacobWT_excludefish11.tif',np.transpose(img_WT,(2,1,0)))
img_WT_std=np.std(img_to_cat[[0,1,2,4,5,6,7,8,9],:,:,:],axis=0)
tifffile.imsave('/mnt/d/ForANTs/MeanJacobWT_std_excludefish11.tif',np.transpose(img_WT_std,(2,1,0)))

In [10]:
img_to_cat=np.zeros([len(np.where(genotype)[0]),1280,1080,251])
for img_nb in range(0,len(np.where(genotype)[0])):
    img=nib.load(warp_list[np.where(genotype)[0][img_nb]].split('1Warp.nii.gz')[0]+'_Jacobian.nii.gz')
    img_to_cat[img_nb,:,:,:]=img.get_fdata()
img_Mut=np.mean(img_to_cat,axis=0)
#tifffile.imsave('/mnt/d/ForANTs/MeanJacobScn1n.tif',np.transpose(img_Mut,(2,1,0)))
img_Mut_std=np.std(img_to_cat,axis=0)
#tifffile.imsave('/mnt/d/ForANTs/MeanJacobScn1n_std.tif',np.transpose(img_Mut_std,(2,1,0)))

In [16]:
img_results_pos=img_Mut
img_results_pos[np.where(img_Mut<(img_WT+abs(img_WT_std)))]=0
tifffile.imsave('/mnt/d/ForANTs/SignificantJacobSCN_SD_pos.tif',np.transpose(img_results_pos,(2,1,0)))

In [17]:
img_results_neg=img_Mut
img_results_neg[np.where(img_Mut>(img_WT-abs(img_WT_std)))]=0
tifffile.imsave('/mnt/d/ForANTs/SignificantJacobSCN_SD_neg.tif',np.transpose(img_results_neg,(2,1,0)))

In [70]:
abs(-1)

1